# Task Management and Dependency Resolution

Multi-agent task orchestration with dependency resolution and sequential execution.

In [ ]:
import sys
sys.path.append('../../../src')

from strands import Agent
from strands.models.ollama import OllamaModel

In [ ]:
model = OllamaModel(
    host="http://localhost:11434",
    model_id="granite4:tiny-h",
    temperature=0.1,
    top_p=0.9
)

In [ ]:
financial_agent = Agent(
    model=model,
    system_prompt="You extract and structure financial data from reports."
)

analyst_agent = Agent(
    model=model,
    system_prompt="You identify trends in financial time series."
)

report_agent = Agent(
    model=model,
    system_prompt="You create clear financial analysis reports."
)

In [ ]:
tasks = {
    "data_extraction": {
        "description": "Extract key financial data from the quarterly report",
        "status": "pending",
        "agent": financial_agent,
        "dependencies": []
    },
    "trend_analysis": {
        "description": "Analyze trends in the extracted data",
        "status": "pending",
        "agent": analyst_agent,
        "dependencies": ["data_extraction"]
    },
    "report_generation": {
        "description": "Generate comprehensive analysis report",
        "status": "pending",
        "agent": report_agent,
        "dependencies": ["trend_analysis"]
    }
}

In [ ]:
def execute_workflow(tasks, input_data):
    results = {}
    completed = set()
    
    while len(completed) < len(tasks):
        for task_id, task in tasks.items():
            if task_id in completed or task["status"] != "pending":
                continue
                
            if all(dep in completed for dep in task["dependencies"]):
                print(f"Executing task: {task_id}")
                
                context = f"Task: {task['description']}\n"
                if task["dependencies"]:
                    context += "Previous results:\n"
                    for dep in task["dependencies"]:
                        context += f"- {dep}: {results.get(dep, 'No result')}\n"
                
                context += f"\nInput data: {input_data}"
                
                result = task["agent"](context)
                results[task_id] = str(result)
                task["status"] = "completed"
                completed.add(task_id)
                
                print(f"Completed: {task_id}\n")
    
    return results

In [ ]:
input_data = """
Q3 2024 Financial Report:
Revenue: $2.5M (+15% YoY)
Expenses: $1.8M (+8% YoY)
Net Income: $700K (+35% YoY)
Cash Flow: $900K (+20% YoY)
"""

results = execute_workflow(tasks, input_data)

In [ ]:
for task_id, result in results.items():
    print(f"=== {task_id.upper()} ===")
    print(result)
    print()